In [55]:
import os
import polars as pl

def get_df(gpu_name, folder: str):
    paths = os.listdir(folder)
    df_list = []
    for path in paths:
        for task_type in task_types:
            for pcs in precisions:            
                if pcs in path and task_type in path and '1_gpus' in path:
                    srs = pl.read_csv(os.path.join(folder, path)).mean().with_columns(
                        pl.all().round(2),
                        pl.lit(gpu_name).alias('gpu_name'),
                        pl.lit(task_type).alias('task_type'),
                        pl.lit(pcs).alias('precision'),
                        )                                   
                    df_list.append(srs)
    return pl.concat(df_list).sort(by=['task_type', 'precision'], descending=True)
                

my_gpu_name = "6800XT"
my_folder = "result"
other_gpu_names = ["2060", #"2080TI", 
                   "2080ti(2)", "3090", "4060Ti16GB"]
task_types = ['train', 'inference']
precisions = ['double', 'float', 'half']
exclude_cols = ['gpu_name', 'task_type', 'precision']

my_df = get_df(my_gpu_name, my_folder)
print(my_df)

shape: (6, 37)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬──────────┬───────────┬───────────┐
│ mnasnet0_ ┆ mnasnet0_ ┆ mnasnet1_ ┆ mnasnet1_ ┆ … ┆ shufflene ┆ gpu_name ┆ task_type ┆ precision │
│ 5         ┆ 75        ┆ 0         ┆ 3         ┆   ┆ t_v2_x2_0 ┆ ---      ┆ ---       ┆ ---       │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ str      ┆ str       ┆ str       │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆          ┆           ┆           │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪══════════╪═══════════╪═══════════╡
│ 25.7      ┆ 29.97     ┆ 33.77     ┆ 42.02     ┆ … ┆ 25.01     ┆ 6800XT   ┆ train     ┆ half      │
│ 31.87     ┆ 37.31     ┆ 41.58     ┆ 52.79     ┆ … ┆ 30.1      ┆ 6800XT   ┆ train     ┆ float     │
│ 95.09     ┆ 109.37    ┆ 121.58    ┆ 143.09    ┆ … ┆ 99.83     ┆ 6800XT   ┆ train     ┆ double    │
│ 5.84      ┆ 7.31      ┆ 6.82      ┆ 8.68      ┆ … ┆ 6.66      ┆ 6800XT   ┆

In [57]:
for other in other_gpu_names:
    other_folder = 'results/' + other
    other_df = get_df(other, other_folder)
    extra = other_df[exclude_cols]
    comp_df = (other_df.select(pl.exclude(exclude_cols)) / my_df.select(pl.exclude(exclude_cols)))
    comp_df = comp_df.with_columns(
        pl.all().mean().round().name.suffix('_mean'),
        pl.lit(comp_df.mean_horizontal()).alias('mean_horizontal'),
    )
    comp_df = comp_df.hstack(extra).with_columns(pl.col('gpu_name') + f' / {my_gpu_name}')
    print(f'{other} / {my_gpu_name} {comp_df = }')
    

2060 / 6800XT comp_df = shape: (6, 68)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ mnasnet0_ ┆ mnasnet0_ ┆ mnasnet1_ ┆ mnasnet1_ ┆ … ┆ mean_hori ┆ gpu_name  ┆ task_type ┆ precisio │
│ 5         ┆ 75        ┆ 0         ┆ 3         ┆   ┆ zontal    ┆ ---       ┆ ---       ┆ n        │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ str       ┆ str       ┆ ---      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆           ┆           ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 1.668093  ┆ 1.597598  ┆ 1.464614  ┆ 1.323417  ┆ … ┆ 2.373916  ┆ 2060 /    ┆ train     ┆ half     │
│           ┆           ┆           ┆           ┆   ┆           ┆ 6800XT    ┆           ┆          │
│ 1.341073  ┆ 1.397481  ┆ 1.458393  ┆ 1.318053  ┆ … ┆ 2.296023  ┆ 2060 /    ┆ train     ┆ float    │
│           ┆           ┆           ┆           ┆   